# Simple data reduction

We will see here the basic steps to:
<ol>
    <li><a href="#sec_read">open</a> a <a href="https://fits.gsfc.nasa.gov/fits_documentation.html">FITS</a> image,</li>
    <li><a href="#sec_dark">create a master DARK frame</a>,</li>
    <li><a href="#sec_flat">create a master FLAT frame</a>,</li>
    <li><a href="#sec_sci">reduce a scientific frame</a> (DARK subtraction, FLAT field correction).</li>
</ol>

Reminder:
$ Reduced = \frac{Image - Dark}{Flat}$, where $<Flat> = 1$ to preserve the count level of the original frame.
<!-- 07 47 20.33107 	+13 14 9.1090 -->


<a id="sec_read"></a>
# 1. Inspection of a DARK frame

First, let's open and inspect a DARK frame (in <a href="https://fits.gsfc.nasa.gov/fits_documentation.html">FITS</a> format) taken at the telescope (<a href="https://www.oca.eu/fr/c2pu-accueil">C2PU</a>).

In [ ]:
from astropy.io import fits, ascii

filename = 'darks/DARK_20151128T140848418_DK_R_0300s000_000000.fits'
hdu = fits.open(filename)
type(hdu)

A <a href="https://fits.gsfc.nasa.gov/fits_documentation.html">FITS</a> file contains the <kbd>data</kbd> itself, and a <kbd>header</kbd> with all the meta-data (describing how, when the image was taken, <em>etc</em>). Together refered as Header Data Unit (HDU).
<strong>NB</strong>: <a href="https://fits.gsfc.nasa.gov/fits_documentation.html">FITS</a> headers are a simple list of strings, in the format<br>
KEYWORD = VALUE / COMMENT<br>
<br>
We can see here that <kbd>fits.open</kbd> return a HDUList (a list of HDUs) objects. The list has in the present case only <strong>one</strong> element, but some instruments may produce <a href="https://fits.gsfc.nasa.gov/fits_documentation.html">FITS</a> with multiple extensions that can be accessed using different indices. Let's place the first element of <kbd>hdu</kbd> in itself for commodity, and see its header:

In [ ]:
print('Number of elements in hdu:', len(hdu))
hdu = hdu[0]
hdu.header

It is easy to retrieve the values stored in a header:

In [ ]:
telescope = hdu.header.get('TELESCOP').strip()
acq_type = hdu.header.get('ACQTYPE').strip()
epoch = hdu.header.get('DATE-OBS')
exptime = hdu.header.get('EXPTIME')
size_x  = hdu.header.get('NAXIS1')
size_y  = hdu.header.get('NAXIS2')

print(f'{acq_type} ({size_x}x{size_y}) taken with {telescope} on {epoch} ({exptime}s exposure).')

<br>Let's plot the image. We will first estimate its median level and standard deviation to use a convenient color scale. We use here a <a href="https://www.gnu.org/software/gnuastro/manual/html_node/Sigma-clipping.html">sigma clipping</a> method to reject outliers.

As the image is very large (check out the <kbd>NAXIS1</kbd> and <kbd>NAXIS2</kbd> keywords in the header) we will also display a sub-part of the image to see details at the pixel level. 

In [ ]:
from astropy.nddata import Cutout2D
position = (2300,2100)
size = (400, 400)
cutout = Cutout2D(hdu.data, position=position, size=size)

In [ ]:
from astropy.stats import sigma_clipped_stats
import matplotlib.pyplot as plt

bkg_mean, bkg_median, bkg_sigma = sigma_clipped_stats(hdu.data, sigma=3.0)

# Create the figure with two subplots
fig, axs = plt.subplots(1, 2, figsize=(15,7))

# Left plot: full-frame
im = axs[0].imshow(hdu.data, cmap='Greys', origin='lower',
           vmin=bkg_median-2*bkg_sigma, 
           vmax=bkg_median+5*bkg_sigma )
axs[0].set_title('Dark - Full frame')

# Indicate zoom
rect = plt.Rectangle( [position[0]-size[0]/2., position[1]-size[1]/2.], size[0], size[1], edgecolor='b', facecolor='None' )
axs[0].add_patch(rect)
axs[0].text( position[0]+size[0]/2., position[1]+size[1]/2.+40, 'Cutout region', color='b',fontsize='x-large', ha='center')

# Right plot: Cutout
axs[1].imshow(cutout.data, cmap='Greys', origin='lower',
           vmin=bkg_median-2*bkg_sigma, 
           vmax=bkg_median+5*bkg_sigma )
axs[1].set_title('Dark - Crop-out')


# Highlight cosmics
cosmics = [ [70,380], [375, 20], [335,10], [368,275] ]
for i in range(len(cosmics)):
    circle = plt.Circle(cosmics[i], 10, edgecolor='r', facecolor='None')
    axs[1].add_patch(circle)
axs[1].text( 300, 40, 'Cosmics', color='r',fontsize='x-large')

# Single color bar
fig.colorbar(im, ax=axs, shrink=0.6, label='ADU' )

# Set x/y labels for all plots
for axe in axs:
    axe.set_xlabel('X (pixels)')
    axe.set_ylabel('Y (pixels)')

As visible both on the full frame image or the zoomed part, there are some cosmetic effects, such as cosmic rays. We will <strong>combine</strong> several DARK frames to remove these artifacts, and create a clean calibration frame (called master DARK).

<a id="sec_dark"></a>
# 2. Creation of a Master DARK

Let's create a list of all the DARK frames, read them, and store them into a <a href="https://numpy.org/doc/stable/">numpy</a> array.

In [ ]:
import os
import numpy as np

# List DARK frames 
list_dark = os.listdir('darks/')
nb_dark = len(list_dark)

# If you had more DARKs in the directory, with different integration time, you would have to sort them.
# The glob package can be handy...

# Store frames in nparray
darks = np.empty((size_x, size_y, nb_dark))
for i in range(nb_dark):
    print(list_dark[i])
    hdu = fits.open('darks/'+list_dark[i])[0]
    darks[:,:,i] = hdu.data

<br><br>
We then combine all the frames, pixel by pixel. In this simple exemple here, we only have 3 DARK, and will combine them by simple median filtering. To increase the signal to noise ratio of the master DARK, more frames, combined with a <a href="https://www.gnu.org/software/gnuastro/manual/html_node/Sigma-clipping.html">sigma clipping</a> method is recommended.

In [ ]:
master_dark = np.median(darks, axis=2)

In [ ]:
bkg_mean, bkg_median, bkg_sigma = sigma_clipped_stats(master_dark, sigma=3.0)
master_cutout = Cutout2D(master_dark, position=position, size=size)

# Create the figure with four subplots
fig, axs = plt.subplots(2, 2, figsize=(17,15), gridspec_kw={'wspace':0.1, 'hspace':0.02})


# Top-Left plot: DARK frame
im = axs[0,0].imshow(hdu.data, cmap='Greys', origin='lower',
           vmin=bkg_median-2*bkg_sigma, 
           vmax=bkg_median+5*bkg_sigma )
axs[0,0].set_title('Dark - Full frame')


# Top-Right plot: DARK cutout
axs[0,1].imshow(cutout.data, cmap='Greys', origin='lower',
           vmin=bkg_median-2*bkg_sigma, 
           vmax=bkg_median+5*bkg_sigma )
axs[0,1].set_title('Dark - Cutout')



# Bottom-Left plot: Master DARK
axs[1,0].imshow(master_dark, cmap='Greys', origin='lower',
           vmin=bkg_median-2*bkg_sigma, 
           vmax=bkg_median+5*bkg_sigma )
axs[1,0].set_title('Master Dark - Full frame')


# Bottom-Right plot: Master DARK cutout
axs[1,1].imshow(master_cutout.data, cmap='Greys', origin='lower',
           vmin=bkg_median-2*bkg_sigma, 
           vmax=bkg_median+5*bkg_sigma )
axs[1,1].set_title('Master Dark - Cutout')


# Highlight cosmics
cosmics = np.asarray([ [70,380], [375, 20], [335,10], [368,275] ])
for i in range(len(cosmics)):
    circle = plt.Circle(cosmics[i], 10, edgecolor='r', facecolor='None')
    axs[0,1].add_patch(circle)
    circle = plt.Circle(cosmics[i], 10, edgecolor='r', facecolor='None')
    axs[1,1].add_patch(circle)

    
# Single color bar
fig.colorbar(im, ax=axs, shrink=0.6, label='ADU' )


# Indicate zoom
for axe in axs[:,0]:
    rect = plt.Rectangle( [position[0]-size[0]/2., position[1]-size[1]/2.], size[0], size[1], edgecolor='b', facecolor='None' )
    axe.add_patch(rect)

# Set x labels for bottom plots
for axe in axs[1,:]:
    axe.set_xlabel('X (pixels)')
    
# Set y labels for left plots
for axe in axs[:,0]:
    axe.set_ylabel('Y (pixels)')

As visible in the figure above, the median filtering has effectively removed many spurious sources, but not all. Most of the black pixels correspond to defectuous pixels (called <em>hot</em> as they are over-sensitive), always located on the same spot (hence not corrected by the median filtering).

They can be identified from their deviation to the Gaussian distribution followed by the regular pixels. We can set a <kbd>threshold</kbd> above which pixels are considered as hot.

In [ ]:
threshold = 5
hot_pixel = np.where( master_dark > bkg_median+threshold*bkg_sigma)

In [ ]:
adu_range = [bkg_median-8*bkg_sigma,bkg_median+8*bkg_sigma]
plt.hist(master_dark.flatten(), range=adu_range, bins=100, label='Pixels' )
plt.hist(master_dark[hot_pixel].flatten(), range=adu_range, bins=100, label='Hot pixels' )
plt.xlabel('ADU')
plt.ylabel('Counts')
plt.yscale('log')
plt.legend(loc='upper left')

For the sake of cosmetic, we will replace these hot pixels by the median value of the surrounding pixels (taken in a 5x5 window for instance). To measure the photometry, the best is however to flag these pixels to <strong>do not count them in the analysis</strong>.

In [ ]:
from scipy.ndimage import median_filter
smoothed = median_filter( master_dark, (5,5) )
master_dark[hot_pixel] = smoothed[hot_pixel]

In [ ]:
bkg_mean, bkg_median, bkg_sigma = sigma_clipped_stats(master_dark, sigma=3.0)
master_cutout = Cutout2D(master_dark, position=position, size=size)

# Create the figure with four subplots
fig, axs = plt.subplots(2, 2, figsize=(17,15), gridspec_kw={'wspace':0.1, 'hspace':0.02})


# Top-Left plot: DARK frame
im = axs[0,0].imshow(hdu.data, cmap='Greys', origin='lower',
           vmin=bkg_median-2*bkg_sigma, 
           vmax=bkg_median+5*bkg_sigma )
axs[0,0].set_title('Dark - Full frame')

# Top-Right plot: DARK cutout
axs[0,1].imshow(cutout.data, cmap='Greys', origin='lower',
           vmin=bkg_median-2*bkg_sigma, 
           vmax=bkg_median+5*bkg_sigma )
axs[0,1].set_title('Dark - Crop-out')



# Bottom-Left plot: Master DARK
axs[1,0].imshow(master_dark, cmap='Greys', origin='lower',
           vmin=bkg_median-2*bkg_sigma, 
           vmax=bkg_median+5*bkg_sigma )
axs[1,0].set_title('Master Dark - Full frame')

# Bottom-Right plot: Master DARK cutout
axs[1,1].imshow(master_cutout.data, cmap='Greys', origin='lower',
           vmin=bkg_median-2*bkg_sigma, 
           vmax=bkg_median+5*bkg_sigma )
axs[1,1].set_title('Master Dark - Crop-out')

# Single color bar
fig.colorbar(im, ax=axs, shrink=0.6, label='ADU' )

# Indicate zoom
for axe in axs[:,0]:
    rect = plt.Rectangle( [position[0]-size[0]/2., position[1]-size[1]/2.], size[0], size[1], edgecolor='b', facecolor='None' )
    axe.add_patch(rect)

# Set x labels for bottom plots
for axe in axs[1,:]:
    axe.set_xlabel('X (pixels)')
    
# Set y labels for bottom plots
for axe in axs[:,0]:
    axe.set_ylabel('Y (pixels)')


We can save this master DARK (without cosmics nor hot pixels) to disk for later usage.

In [ ]:
hdu.data = master_dark
hdu.writeto('master_dark.fits', overwrite=True)

<a id="sec_flat"></a>
# 3. Creation of a Master FLAT

<strong>YOUR</strong> turn!

Three flat frames are available in the <em>flats</em> directory.
Build a master flat frame and call it <kbd>master_flat</kbd>.

A few reminders:
<ul>
    <li>Flats allow to measure the response of each pixel to light, but contain a dark current too (use the same DARK here for the sake of the exercise, but the <strong>appropriate</strong> DARK must be used for each frame, either FLAT or scientific)</li>
    <li>The average level of each flat taken on the sky may likely be different from any another flat</li>
    <li>Some defectuous pixels are under-sensitive (called <em>dead</em> pixels) and should also be masked out (call their indices <kbd>dead_pixel</kbd>)</li>
</ul>    

<a id="sec_sci"></a>
# 4. Reduction of scientific frames

It is now straightforward to reduce a scientific frame:

In [ ]:
# Read scientific frame
hdu = fits.open('data/17365-1978-VF11_20151125T223627610_SC_R_0300s000_000000.fits')[0]

# Subtract dark and divide by flat
master_dark = fits.open('master_dark.fits')[0].data
master_flat = fits.open('master_flat.fits')[0].data

reduced = ( hdu.data - master_dark ) / master_flat

# Interpolate bad pixels
smoothed = median_filter( reduced, (5,5) )
if 'hot_pixel' in locals():
    print('Cleaning hot pixels')
    reduced[hot_pixel] = smoothed[hot_pixel]
if 'dead_pixel' in locals():
    print('Cleaning dead pixels')
    reduced[dead_pixel] = smoothed[dead_pixel]

In [ ]:
image_cutout = Cutout2D(hdu.data, position=position, size=size)
reduced_cutout = Cutout2D(reduced, position=position, size=size)

# Create the figure with four subplots
fig, axs = plt.subplots(2, 2, figsize=(17,15), gridspec_kw={'wspace':0.1, 'hspace':0.02})


# Top row: Original ADU scale
bkg_mean, bkg_median, bkg_sigma = sigma_clipped_stats(hdu.data, sigma=3.0)

# Top-Left plot: Original frame
im = axs[0,0].imshow(hdu.data, cmap='Greys', origin='lower',
           vmin=bkg_median-2*bkg_sigma, 
           vmax=bkg_median+5*bkg_sigma )
axs[0,0].set_title('Image - Full frame')

# Top-Right plot: Original cutout
axs[0,1].imshow(image_cutout.data, cmap='Greys', origin='lower',
           vmin=bkg_median-2*bkg_sigma, 
           vmax=bkg_median+5*bkg_sigma )
axs[0,1].set_title('Image - Crop-out')

# Top row color bar
fig.colorbar(im, ax=axs[0,:], shrink=0.6, label='ADU' )


# Bottom Row: Reduced pixel scale (normalized to average)
bkg_mean, bkg_median, bkg_sigma = sigma_clipped_stats(reduced, sigma=3.0)

# Bottom-Left plot: Reduced
im = axs[1,0].imshow(reduced, cmap='Greys', origin='lower',
           vmin=bkg_median-2*bkg_sigma, 
           vmax=bkg_median+5*bkg_sigma )
axs[1,0].set_title('Reduced - Full frame')

# Bottom-Right plot: Reduced cutout
axs[1,1].imshow(reduced_cutout.data, cmap='Greys', origin='lower',
           vmin=bkg_median-2*bkg_sigma, 
           vmax=bkg_median+5*bkg_sigma )
axs[1,1].set_title('Reduced - Crop-out')

# Bottom row color bar
fig.colorbar(im, ax=axs[1,:], shrink=0.6, label='Variation' )

# Indicate zoom
for axe in axs[:,0]:
    rect = plt.Rectangle( [position[0]-size[0]/2., position[1]-size[1]/2.], size[0], size[1], edgecolor='b', facecolor='None' )
    axe.add_patch(rect)

# Set x labels for bottom plots
for axe in axs[1,:]:
    axe.set_xlabel('X (pixels)')
    
# Set y labels for bottom plots
for axe in axs[:,0]:
    axe.set_ylabel('Y (pixels)')

Let's write the results, <strong>with its header</strong>.

In [ ]:
hdu_reduced = hdu.copy()
hdu_reduced.data = reduced
hdu_reduced.writeto('1978_VF11-reduced.fits', overwrite=True)

Discuss the similarities and differences between the original frame and its reduced version.
Several points must be checked to assert the quality of the data reduction.
<ul>
    <li>Is the flat-field properly corrected?</li>
    <li>Are defectuous pixels (both dead and hot) properly identified and corrected?</li>
    <li>Is the dark current correctly subtracted?</li>
    <li>Are they signals that appeared in the reduced frames that were not present in the original frame?</li>
</ul>

<a id="sec_real"></a>
# 5. Real life

In the present notebook, we have quickly seen how to read the frames, apply median filtering, and subtract/divide them. There are <strong>many</strong> aspects of the data reduction that <strong>must</strong> be improved from this notebook.

<ol>
    <li>What can be improved by applying another filter to create the master dark or master flat?</li>
    <li>Is it useful to take more calibration data? 1, 2, 3, 4, 5, ...12, 13, ... 100, 1000?</li>
    <li>We identified hot pixels, how can we identify dead pixels?</li>
    <li>Is replacing <em>bad</em> pixels by the median of the surrouding pixels the best thing to do?</li>
    <li>How can we improve the flat frame?</li>
</ol>

<br><br><br><br><br><br><br><br><br>
Solution for the master flat... 
<br>Only read it if you really tried...<br>
<br>
Don't cheat!

In [ ]:
# List flats
list_flat = os.listdir('flats/')
nb_flat = len(list_flat)

# Read and store in array
flats = np.empty((size_x, size_y,nb_flat))
for i in range(nb_flat):
    print(list_flat[i])
    hdu = fits.open('flats/'+list_flat[i])[0]
    flats[:,:,i] = hdu.data / np.median(hdu.data) # Try removing the normalization - What happens?

# Median filtering
master_flat = np.median(flats, axis=2)

# Bad pixel (dead) removal
bkg_mean_flat, bkg_median_flat, bkg_sigma_flat = sigma_clipped_stats(master_flat, sigma=3.0)
dead_pixel = np.where(master_flat < bkg_median_flat-threshold*bkg_sigma_flat)
smoothed_flat = median_filter( master_flat, (5,5) )
master_flat[dead_pixel] = smoothed[dead_pixel]

# Subtract DARK
master_flat = master_flat - master_dark

# Normalization 
master_flat = master_flat/np.mean(master_flat)

# Write flat to disk
hdu.data = master_flat
hdu.writeto('master_flat.fits', overwrite=True)

In [ ]:
bkg_mean_flat, bkg_median_flat, bkg_sigma_flat = sigma_clipped_stats(master_flat, sigma=3.0)
adu_range = [bkg_median_flat-8*bkg_sigma_flat,bkg_median_flat+8*bkg_sigma_flat]
plt.hist(master_flat.flatten(), range=adu_range, bins=100, label='Pixels' )
plt.hist(master_flat[dead_pixel].flatten(), range=adu_range, bins=100, label='Dead pixels' )
plt.xlabel('ADU')
plt.ylabel('Counts')
plt.yscale('log')
plt.legend(loc='upper left')

In [ ]:
master_cutout = Cutout2D(master_flat, position=position, size=size)
cutout = Cutout2D(hdu.data, position=position, size=size)

# Create the figure with four subplots
fig, axs = plt.subplots(2, 2, figsize=(17,15), gridspec_kw={'wspace':0.1, 'hspace':0.02})


# Top row: Original ADU scale
bkg_mean, bkg_median, bkg_sigma = sigma_clipped_stats(hdu.data, sigma=3.0)

# Top-Left plot: FLAT frame
im = axs[0,0].imshow(hdu.data, cmap='Greys', origin='lower',
           vmin=bkg_median-2*bkg_sigma, 
           vmax=bkg_median+5*bkg_sigma )
axs[0,0].set_title('Flat - Full frame')

# Top-Right plot: FLAT cutout
axs[0,1].imshow(cutout.data, cmap='Greys', origin='lower',
           vmin=bkg_median-2*bkg_sigma, 
           vmax=bkg_median+5*bkg_sigma )
axs[0,1].set_title('Flat - Crop-out')

# Top row color bar
fig.colorbar(im, ax=axs[0,:], shrink=0.6, label='ADU' )


# Bottom Row: Master Flat pixel scale (normalized to average)
bkg_mean, bkg_median, bkg_sigma = sigma_clipped_stats(master_flat, sigma=3.0)

# Bottom-Left plot: Master FLAT
im = axs[1,0].imshow(master_flat, cmap='Greys', origin='lower',
           vmin=bkg_median-2*bkg_sigma, 
           vmax=bkg_median+5*bkg_sigma )
axs[1,0].set_title('Master Flat - Full frame')

# Bottom-Right plot: Master DARK cutout
axs[1,1].imshow(master_cutout.data, cmap='Greys', origin='lower',
           vmin=bkg_median-2*bkg_sigma, 
           vmax=bkg_median+5*bkg_sigma )
axs[1,1].set_title('Master Flat - Crop-out')

# Bottom row color bar
fig.colorbar(im, ax=axs[1,:], shrink=0.6, label='Variation' )

# Indicate zoom
for axe in axs[:,0]:
    rect = plt.Rectangle( [position[0]-size[0]/2., position[1]-size[1]/2.], size[0], size[1], edgecolor='b', facecolor='None' )
    axe.add_patch(rect)

# Set x labels for bottom plots
for axe in axs[1,:]:
    axe.set_xlabel('X (pixels)')
    
# Set y labels for bottom plots
for axe in axs[:,0]:
    axe.set_ylabel('Y (pixels)')

# What's that?
issues = [ [380,115] ]
for i in range(len(issues)):
    circle = plt.Circle(issues[i], 20, edgecolor='r', facecolor='None')
    axs[0,1].add_patch(circle)
    axs[0,1].text( 380, 150, 'What is that?', color='r', fontsize='x-large', ha='right')